In [ ]:
# Import packages
from enum import Enum
from pathlib import Path
import psycopg2
import pandas as pd
from IPython.display import Image, Markdown, display
from tqdm import tqdm
import os
from langchain.messages import AIMessage
from langchain_ollama import ChatOllama
from langchain_deepseek import ChatDeepSeek
from pydantic import BaseModel, Field
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.output_parsers import PydanticOutputParser
from langchain.agents import create_agent
from agentic_ai_tools import PaymentReferenceSearch, AccessAccountsReceivable, AccessPayments
import datetime as dt
from langgraph.checkpoint.memory import InMemorySaver
from langchain.agents.structured_output import ToolStrategy
import urllib.parse
from sqlalchemy import create_engine
import re

In [12]:
#load_dotenv(find_dotenv())
DB_NAME=os.getenv('DB_NAME')
USERNAME=os.getenv('USERNAME')
PASSWORD=urllib.parse.quote(os.getenv('PASSWORD'))
HOSTNAME=os.getenv('HOSTNAME')
PORT=os.getenv('PORT')
FUZZ_RATIO_THRESHOLD = os.getenv('FUZZ_RATIO_THRESHOLD')
# Creating conneciton to database
engine = create_engine(f'postgresql+psycopg2://{USERNAME}:{PASSWORD}@{HOSTNAME}/{DB_NAME}')




def invoice_number_match(dataframe, reference_component, threshold):
    invoice_match=False
    for idx, row in dataframe.iterrows():
        fuzzy_ratio = fuzz.ratio(str(row['invoice_number']), reference_component)
        #print(fuzzy_ratio)
        if fuzzy_ratio is not None and fuzzy_ratio>threshold:
            invoice_match = True
            print("Invoice Number Match!")
            return idx, invoice_match

def customer_number_match(dataframe, reference_component, threshold):
    customer_match=False
    for idx, row in dataframe.iterrows():
        fuzzy_ratio = fuzz.ratio(str(row['customer_number']), reference_component)
        if fuzzy_ratio is not None and fuzzy_ratio>threshold:
            customer_match=True
            print("Customer Match!")
            return idx, customer_match

def amount_number_match(dataframe, reference_component, threshold):
    is_match=0
    for idx, row in dataframe.iterrows():
        fuzzy_ratio = fuzz.ratio(str(row['amount']), reference_component)
        if fuzzy_ratio is not None and fuzzy_ratio>threshold:
            is_match=1
            print("Amount number match!")
            return idx, is_match

def fill_details(in_idx,index,payments_dataframe, accounts_receivables_dataframe):
    if accounts_receivables_dataframe.loc[in_idx,'payment'] == None:
        accounts_receivables_dataframe.loc[in_idx,'payment'] = payments_dataframe.loc[index,'payment_amount']
    else:
        accounts_receivables_dataframe.loc[in_idx,'payment'] += payments_dataframe.loc[index,'payment_amount']
    accounts_receivables_dataframe.loc[in_idx,'payment_date'] = payments_dataframe.loc[index,'payment_date']
    accounts_receivables_dataframe.loc[in_idx, 'payment_id'] = payments_dataframe.loc[index,'transaction_id']
    return accounts_receivables_dataframe


def AccessAccountsReceivable():
    """Function to access the accounts receivables data table in Postgres"""
    accounts_receivables = pd.read_sql("SELECT * FROM accounts_receivable", engine)
    return accounts_receivables.to_dict()

def AccessPayments():
    """Function to access the payments received data table in postgress"""
    payments = pd.read_sql("SELECT * FROM payments", engine)
    return payments.to_dict()


In [13]:
ar = AccessAccountsReceivable()
pr = AccessPayments()

In [14]:
def PaymentReferenceSearch(ar: dict,  pr: dict) -> dict:
    """Fuzzy search of payment reference string for a similarity check of each string.
    The payments_dataframe has the output of AccessPayments as input. The accounts_receivables dataframe
    has the outut of AccessAccountsReceivable as input.

    This function should only be used after the tools AccessAccountsReceivable and AccesssPayments have been used.

    Function returns the updated Accounts Receivable and payments dataframe.
    """
    fuzz_threshold = int(FUZZ_RATIO_THRESHOLD)
    payments_dataframe = pd.DataFrame(pr, index=[0,len(pr)])
    accounts_receivables_dataframe = pd.DataFrame(ar, index=[0,len(ar)])
    # Add additional column for payments_dataframe to categorise if payment has been matched or not.
    #payments_dataframe['matched'] = False

    try:
        for index, row in payments_dataframe.iterrows():
            # Try first just the payment reference information
            print(str(row['payment_reference']))
            pattern = r"\s"
            string_list=re.split(pattern, str(row['payment_reference']))
            print(string_list)
            customer_match = False
            invoice_match = False
            for component in string_list:
                component = component.strip()
                print(f"Element: {component}")
                if component == None:
                    pass
                else:
                    # for i in range(1):
                    #     print(f"ROUND:{i}")

                    if customer_match:
                        # Invoice number match
                        print("Starting Invoice Number Match")
                        try:
                            in_idx, invoice_match = invoice_number_match(dataframe=accounts_receivables_dataframe,
                                    reference_component=component,
                                    threshold=fuzz_threshold)
                            
                        except Exception as e:
                            print(f"Error Invoice Match: {e}")
                            pass
                    
                    else:
                        
                        print("Starting Customer Number Match")
                        try:
                            cs_idx, customer_match = customer_number_match(dataframe=accounts_receivables_dataframe,
                                                reference_component=component,
                                                threshold=fuzz_threshold)

                        except Exception as e:
                            print(f"Error Customer Match: {e}")
                            pass
                        
                    if invoice_match == True and customer_match == True:
                        print("Invoice and Customer matched!")
                        payments_dataframe.loc[index,'matched'] = True
                        fill_details(in_idx,index, payments_dataframe,accounts_receivables_dataframe)
                        break
                     
                    else:
                      payments_dataframe.loc[index,'matched'] = False
                        

                
    except Exception as e:
        print(f"OUTER ERROR: {e}")
        pass

    return accounts_receivables_dataframe.to_dict()

In [15]:
PaymentReferenceSearch(pr=pr, ar=ar)

12038 - 1 - We'll pay the rest later
['12038', '-', '1', '-', "We'll", 'pay', 'the', 'rest', 'later']
Element: 12038
Starting Customer Number Match
Error Customer Match: name 'fuzz' is not defined
Element: -
Starting Customer Number Match
Error Customer Match: name 'fuzz' is not defined
Element: 1
Starting Customer Number Match
Error Customer Match: name 'fuzz' is not defined
Element: -
Starting Customer Number Match
Error Customer Match: name 'fuzz' is not defined
Element: We'll
Starting Customer Number Match
Error Customer Match: name 'fuzz' is not defined
Element: pay
Starting Customer Number Match
Error Customer Match: name 'fuzz' is not defined
Element: the
Starting Customer Number Match
Error Customer Match: name 'fuzz' is not defined
Element: rest
Starting Customer Number Match
Error Customer Match: name 'fuzz' is not defined
Element: later
Starting Customer Number Match
Error Customer Match: name 'fuzz' is not defined
12990  - Invoice Nr. 3
['12990', '', '-', 'Invoice', 'Nr.', 

{'invoice_number': {0: 1, 9: 10},
 'date': {0: datetime.date(2025, 5, 1), 9: datetime.date(2025, 5, 2)},
 'customer_name': {0: 'Planet Express',
  9: 'Cookieville Minimum-Security Orphanarium'},
 'customer_number': {0: 12038, 9: 11900},
 'amount': {0: 50000.0, 9: 80000.0},
 'due_date': {0: datetime.date(2025, 5, 2), 9: datetime.date(2025, 5, 2)},
 'payment': {0: None, 9: None},
 'payment_date': {0: None, 9: None},
 'payment_id': {0: None, 9: None}}